*upd. 14-feb-2026*
Se seleccionaron tres estaciones para hacer una evaluación preliminar de los índices SPEI y SPI. Estas están distribuidas en el estado de Jalisco, específicamente, en la zona costa (Puerto Vallarta), centro (Guadalajara) y altos (Lagos Moreno). En esta primera parte del script, se hace la limpieza y análisis exploratorio de los CSV descargados del SMN.

In [2]:
#se importan las librerias
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

##### Limpieza de los datos
1. Se usa `skiprows`y `usecols`para leer desde el principio solo la información necesaria
2. Se convierten los valores con la etiqueta NULOS a *NaN*
3. Se elimina la fila que contiene strings con las unidades para que no estorben en futuros cálculos

In [44]:
def carga_smn(id_estac):
    ruta = f"../01_DATA/raw/dia{id_estac}.txt"

    #Leer los archivos evitando los metadatos para que no estorben al hacer lo cálculos
    df = pd.read_csv(ruta, sep='\\s+', na_values=['NULO'], usecols = ('FECHA', 'PRECIP','TMAX', 'TMIN'),skiprows=19)
    
    #Se elimina la fila de las unidades por el mismo motivo anterior
    df = df.drop(index=0).reset_index(drop=True)
        
    #poner columnas en el formato adecuado de variable
    df['FECHA'] = pd.to_datetime(df['FECHA'])
    cols_df = ['PRECIP', 'TMAX', 'TMIN']
    for i in cols_df:
        df[i] = pd.to_numeric(df[i], errors = 'coerce')
    return df

# ---- Para agilizar. se usa un bucle
estacs = ["14339","14067", "14024", #costa
          "14066", "14065", "14169", #centro
          "14084", "14083"] #altos

# se hace un diccionario para que no se sobreescriban los archivos
estacs_leidas = {}

for id_estac in estacs:
    resultado = carga_smn(id_estac)
    estacs_leidas[id_estac] = resultado

In [47]:
print(estacs_leidas['14339'].head())
print(estacs_leidas['14339'].tail())

       FECHA  PRECIP  TMAX  TMIN
0 1980-06-15     NaN   NaN   NaN
1 1980-06-16     0.0  31.0  12.0
2 1980-06-17     0.0  32.0  18.0
3 1980-06-18     0.0  34.0  18.0
4 1980-06-19     0.0  32.0  18.0
           FECHA  PRECIP  TMAX  TMIN
16180 2026-01-30     NaN   NaN   NaN
16181 2026-02-01     9.2   NaN   NaN
16182 2026-02-02     NaN   NaN   NaN
16183 2026-02-04     0.0   NaN   NaN
16184 2026-02-05     NaN   NaN   NaN


##### Control de calidad de los datos

**Límites físicos / outliers**
* Consistencia térmica: en caso de que Tmax < Tmin, se eliminarán ambos valores
* Validación de mínimo: Prcp. no puede ser negativa :. si es negativo se elimina el valor

**Estandarización de las fechas**

In [55]:
def qc_logico(df):   
    # 1. Consistencia Térmica: La Máxima no puede ser menor a la Mínima
    # Identificamos errores de captura/digitalización
    mask_temp_error = df['TMIN'] > df['TMAX']
    
    # Si TMIN > TMAX, invalidamos ambos registros asignando NaN
    df.loc[mask_temp_error, ['TMAX', 'TMIN']] = np.nan
    
    # 2. Límites Físicos: Valores negativos imposibles
    # La precipitación >= 0
    df.loc[df['PRECIP'] < 0, 'PRECIP'] = np.nan
    
    # --- INDEXACIÓN CON FECHAS ---
    # 3. Asegurar que FECHA sea tipo datetime
    df['FECHA'] = pd.to_datetime(df['FECHA'], errors='coerce')
    
    # 4. Eliminar duplicados de fecha antes de indexar
    df = df.drop_duplicates(subset=['FECHA'])
    
    # 5. Establecer FECHA como índice
    df = df.set_index('FECHA')
    
    # 6. Ordenar cronológicamente
    df = df.sort_index()
    
    # Reporte de limpieza para seguimiento
    # Usamos len(mask_temp_error) o el ID si lo tuviéramos para identificar
    print(f"QC: {mask_temp_error.sum()} errores térmicos corregidos. Total: {len(df)} registros.")
    
    return df

# --- BUCLE PARA PROCESAR EL DICCIONARIO ---

# Creamos un nuevo diccionario para los datos limpios
estaciones_qc = {}

for id_estac, df_raw in estacs_leidas.items():
    print(f"Procesando QC para Estación {id_estac}...")
    # Aplicamos la función y guardamos el resultado
    estaciones_qc[id_estac] = qc_logico(df_raw)

# Ahora puedes acceder a cualquier estación limpia así:
# estaciones_qc['14002'].head()

Procesando QC para Estación 14339...
QC: 0 errores térmicos corregidos. Total: 16185 registros.
Procesando QC para Estación 14067...
QC: 0 errores térmicos corregidos. Total: 23706 registros.
Procesando QC para Estación 14024...
QC: 0 errores térmicos corregidos. Total: 23453 registros.
Procesando QC para Estación 14066...
QC: 0 errores térmicos corregidos. Total: 31265 registros.
Procesando QC para Estación 14065...
QC: 0 errores térmicos corregidos. Total: 23910 registros.
Procesando QC para Estación 14169...
QC: 0 errores térmicos corregidos. Total: 26144 registros.
Procesando QC para Estación 14084...
QC: 0 errores térmicos corregidos. Total: 29627 registros.
Procesando QC para Estación 14083...
QC: 0 errores térmicos corregidos. Total: 23423 registros.


##### Integridad de la serie
**Faltantes**
* Calcular cantidad de datos faltantes
* se considerarán las estaciones que contienen más del 90% de los datos
* Calidad Mensual: Que no haya meses con más de 5 días faltantes (estos se vuelven NaN).

In [56]:
def integridad_temporal_masiva(diccionario_estaciones):
    # 1. Validar que el diccionario no esté vacío
    if not diccionario_estaciones:
        print("El diccionario de estaciones está vacío.")
        return {}

    # 2. Identificar la fecha mínima y máxima global de TODO el set de Jalisco
    # Recorremos todos los DataFrames para encontrar los extremos temporales
    fecha_min_global = min([df.index.min() for df in diccionario_estaciones.values()])
    fecha_max_global = max([df.index.max() for df in diccionario_estaciones.values()])
    
    # 3. Crear el calendario maestro diario (sin saltos)
    calendario_maestro = pd.date_range(start=fecha_min_global, end=fecha_max_global, freq='D')
    
    estaciones_normalizadas = {}
    
    # 4. Bucle para reindexar cada estación con el calendario maestro
    for id_est, df in diccionario_estaciones.items():
        # .reindex() es la clave: inserta filas con NaN en las fechas faltantes
        df_norm = df.reindex(calendario_maestro)
        df_norm.index.name = 'FECHA'
        
        # Marcamos qué días son "rellenados" (útil para auditoría de datos)
        # Si PRECIP es NaN, asumimos que es un dato faltante insertado ahora
        df_norm['MISSING_DAY'] = df_norm['PRECIP'].isna().astype(int)
        
        estaciones_normalizadas[id_est] = df_norm
        print(f"Estación {id_est}: Calendario normalizado.")
    
    print(f"\n--- NORMALIZACIÓN GLOBAL FINALIZADA ---")
    print(f"Rango de estudio Jalisco: {fecha_min_global.date()} a {fecha_max_global.date()}")
    print(f"Longitud de la serie: {len(calendario_maestro)} días.")
    
    return estaciones_normalizadas

# --- EJECUCIÓN ---

# Usamos el diccionario 'estaciones_qc' que generamos en el Bloque 2
estaciones_listas = integridad_temporal_masiva(estaciones_qc)

# Ejemplo de verificación:
#print(estaciones_listas['14002'].head())

Estación 14339: Calendario normalizado.
Estación 14067: Calendario normalizado.
Estación 14024: Calendario normalizado.
Estación 14066: Calendario normalizado.
Estación 14065: Calendario normalizado.
Estación 14169: Calendario normalizado.
Estación 14084: Calendario normalizado.
Estación 14083: Calendario normalizado.

--- NORMALIZACIÓN GLOBAL FINALIZADA ---
Rango de estudio Jalisco: 1882-04-01 a 2026-02-05
Longitud de la serie: 52541 días.


##### Guardar lo anterior como CSV

In [57]:
def exportar_rango_especifico(diccionario_normalizado, carpeta_salida, fecha_inicio, fecha_fin):
    if not os.path.exists(carpeta_salida):
        os.makedirs(carpeta_salida)

    for id_est, df in diccionario_normalizado.items():
        # 1. Filtrar el DataFrame usando el índice (que es la FECHA)
        # .loc es inclusivo en ambos extremos
        try:
            df_filtrado = df.loc[fecha_inicio : fecha_fin]
            
            if df_filtrado.empty:
                print(f"Estación {id_est}: Sin datos en el rango {fecha_inicio} a {fecha_fin}. Saltando...")
                continue

            # 2. Guardar el archivo filtrado
            nombre_archivo = f"SMN_{id_est}_{fecha_inicio[:4]}_{fecha_fin[:4]}.csv"
            ruta_completa = os.path.join(carpeta_salida, nombre_archivo)
            
            df_filtrado.to_csv(ruta_completa, sep=',', encoding='utf-8', index=True)
            print(f"Estación {id_est}: Guardada con éxito ({len(df_filtrado)} días).")
            
        except Exception as e:
            print(f"Error al filtrar la estación {id_est}: {e}")

    print(f"\n--- PROCESO FINALIZADO ---")

# --- EJECUCIÓN ---
# Define tus fechas de interés (ejemplo: periodo de 30 años)
inicio = '1991-01-01'
fin = '2020-12-31'
carpeta_proyectos = "../01_DATA/SMN_limpio"

exportar_rango_especifico(estaciones_listas, carpeta_proyectos, inicio, fin)

Estación 14339: Guardada con éxito (10958 días).
Estación 14067: Guardada con éxito (10958 días).
Estación 14024: Guardada con éxito (10958 días).
Estación 14066: Guardada con éxito (10958 días).
Estación 14065: Guardada con éxito (10958 días).
Estación 14169: Guardada con éxito (10958 días).
Estación 14084: Guardada con éxito (10958 días).
Estación 14083: Guardada con éxito (10958 días).

--- PROCESO FINALIZADO ---


# Selección de estaciones
Las estaciones que se trabajaran son las que cumplan los siguientes criterios
a) tienen el 90% de registros (datos faltantes < 10%)
b) no pueden tener cinco registro nulos seguidos 

**Se eligió como periodo de 1991 a 2020 (30 años**)

In [61]:
# --- CONFIGURACIÓN DE RUTAS ---
PATH_ENTRADA = "../01_DATA/SMN_limpio/"
PATH_SALIDA = "../01_DATA/for_indices"
FECHA_INICIO = "1991-01-01"
FECHA_FIN = "2020-12-31"

def tiene_rachas_nan(serie, max_consecutivos=5):
    # Creamos una serie booleana (True donde es NaN)
    es_nan = serie.isna()
    # Identificamos bloques consecutivos de True
    # (Comparamos el valor con el anterior para marcar cambios de bloque)
    bloques = es_nan.ne(es_nan.shift()).cumsum()
    # Contamos el tamaño de cada bloque de NaNs
    tamaño_bloques = es_nan.groupby(bloques).transform('sum')
    # Si algún bloque de NaNs es >= al límite, devolvemos True
    return (tamaño_bloques >= max_consecutivos).any()

def filtrar_estaciones_estricto():
    archivos = [f for f in os.listdir(PATH_ENTRADA) if f.endswith('.csv')]
    aptas = 0
    
    print(f"Iniciando filtrado estricto de {len(archivos)} estaciones...")

    for archivo in archivos:
        df = pd.read_csv(os.path.join(PATH_ENTRADA, archivo), 
                         index_col='FECHA', parse_dates=True)
        
        # 1. Recorte temporal
        df_p = df.loc[FECHA_INICIO : FECHA_FIN]
        
        # 2. Validación de existencia de datos
        if df_p.empty or len(df_p) < 1000: # Filtro básico de seguridad
            continue

        # --- CRITERIOS DE CALIDAD ---
        
        # A. Máximo 10% de faltantes totales
        pct_nan = (df_p['PRECIP'].isna().sum() / len(df_p)) * 100
        
        # B. No más de 5 días SEGUIDOS con NaN
        racha_nan = tiene_rachas_nan(df_p['PRECIP'], max_consecutivos=15)
        
        # C. No más de 5 días faltantes en UN MES (alternos o no)
        # Esto asegura que cada mes sea representativo para el SPI
        meses_invalidos = (df_p['PRECIP'].isna().resample('MS').sum() > 15).any()

        # --- DECISIÓN FINAL ---
        if pct_nan <= 20 and not racha_nan and not meses_invalidos:
            ruta_destino = os.path.join(PATH_SALIDA, archivo)
            df_p.to_csv(ruta_destino)
            aptas += 1
            print(f"✅ {archivo}: APTO (Faltantes: {pct_nan:.1f}%)")
        else:
            razon = []
            if pct_nan > 20: razon.append(f"Total NaNs > 20% ({pct_nan:.1f}%)")
            if racha_nan: razon.append("Racha >= 15 días NaN")
            if meses_invalidos: razon.append("Meses con 15 5 NaNs")
            print(f"❌ {archivo}: Descartado por {', '.join(razon)}")

    print(f"\n--- FILTRADO TERMINADO ---")
    print(f"Estaciones finales en 'for_indices': {aptas}")

filtrar_estaciones_estricto()

Iniciando filtrado estricto de 8 estaciones...
❌ SMN_14024_1991_2020.csv: Descartado por Racha >= 15 días NaN, Meses con 15 5 NaNs
❌ SMN_14065_1991_2020.csv: Descartado por Racha >= 15 días NaN, Meses con 15 5 NaNs
✅ SMN_14066_1991_2020.csv: APTO (Faltantes: 0.1%)
❌ SMN_14067_1991_2020.csv: Descartado por Racha >= 15 días NaN, Meses con 15 5 NaNs
❌ SMN_14083_1991_2020.csv: Descartado por Racha >= 15 días NaN, Meses con 15 5 NaNs
❌ SMN_14084_1991_2020.csv: Descartado por Racha >= 15 días NaN, Meses con 15 5 NaNs
❌ SMN_14169_1991_2020.csv: Descartado por Racha >= 15 días NaN, Meses con 15 5 NaNs
✅ SMN_14339_1991_2020.csv: APTO (Faltantes: 0.1%)

--- FILTRADO TERMINADO ---
Estaciones finales en 'for_indices': 2
